In [29]:
from google import genai
from google.genai import types
import base64
import json
import os
from dotenv import load_dotenv

import unicodedata
import re

In [30]:
load_dotenv()
project_name = os.getenv('PROJECT_NAME')

The story id, from 1 to 13 according to the 13 Ainu Kamuy Yukars translated by Chiri Yukie. The Yukar ID starts at 1. Chiri's Preface is 0.

In [ ]:
start_at = 1
end_at = 13

In [32]:
system_instruction_prompt = """You are a professional translator. You know Japanese, English and Chinese. You can translate Japanese into either Chinese or English."""

client = genai.Client(
      vertexai=True,
      project=project_name,
      location="us-central1",
)

model = "gemini-2.0-flash-001"

generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=system_instruction_prompt)],
  )

In [33]:
translation_prompt = "Translate the following text from Japanese to English. The original text is footnote. Keep the original meanings. For the text which are not in Japanese, keep the original form."


In [34]:
# read the content page of Japanese translation and get the Japanese translated title
with open("Chiri_Japanese_Translation/content.txt", "r", encoding="utf8") as f:
    japanese_content = f.read()
    japanese_content = unicodedata.normalize('NFKC', japanese_content)


s=re.split(r'\n\n', japanese_content)
japanese_titles = re.split(r'\n', s[1])

In [35]:
# read the content page of Ainu original text and get the original title
with open("original_Ainu_text/content.txt", "r", encoding="utf8") as f:
    ainu_content = f.read()
    ainu_content = unicodedata.normalize('NFKC', ainu_content)


s=re.split(r'\n\n', ainu_content)
ainu_titles = re.split(r'\n', s[1])

In [36]:
# Read the markdown template for writing the Chinese translations to Markdown file

#read in the template
with open("templates/raw_output_footnote_md_template", "r", encoding="utf8") as f:
    md_template = f.read()
    md_template = unicodedata.normalize('NFKC', md_template)

In [37]:
def generate(client: genai.Client,generate_content_config :types.GenerateContentConfig,model :str, /,input_text :str, prompt :str):

    text_full_prompt = text1 = types.Part.from_text(text=f"{prompt}\n\n{input_text}")

    output = ""

    contents = [
      types.Content(
        role="user",
        parts=[
          text_full_prompt
        ]
      )
    ]

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        output += chunk.text

    return output

def get_output_file_name_key(title :str):
    # setup the output file name
    s = title.split()
    md_name_part = s[0]

    name_2nd_part = ""

    for text in s:
        if text.startswith('“'):
            name_2nd_part = text.replace('“', '').replace('”', '')
        
    md_name_part += "_" + name_2nd_part

    return md_name_part



In [38]:
# process the translation in batch mode

for song_no in range(start_at, end_at + 1):
    with open(f"Chiri_footnotes/footnote_story_{song_no}.txt", "r", encoding="utf8") as f:
        japanese_text = f.read()
        japanese_text = unicodedata.normalize('NFKC', japanese_text)

        footnote_translation = generate(client,generate_content_config,model,input_text=japanese_text,prompt=translation_prompt)

        md_output = md_template.format(translated_language="English", ainu_title=ainu_titles[song_no - 1],
                               prompt=translation_prompt,
                               japanese_title=japanese_titles[song_no], input_japanese = japanese_text,
                               output=footnote_translation)
        
        md_name_part = get_output_file_name_key(ainu_titles[song_no - 1])

        with open(f"LLM_prompts_and_raw_translations_footnotes/English_Translation/{song_no}_{md_name_part}_to_English.md", "w", encoding="utf8") as f:
            f.write(md_output)



Okay, here are the English translations of the footnotes, preserving the original meanings and keeping non-Japanese text as is:

(1) In the old days, when a boy got a little older, they would make and give him a small bow and arrow. The child would play by shooting at trees and birds, unknowingly becoming skilled in archery.
ak...... archery, shinot is play, ponai is small arrow.

(2) shiktumorke...... the look in one's eyes.
It is said that if you want to know a person's true nature, the best way is to look at their eyes, and they would be scolded if they looked around restlessly.

(3) achikara...... means "dirty."

(4) It is said that birds and animals are shot down by people because they want the arrows made by people, and they are taking the arrows.

(5) kotankorkamui...... a god who possesses a country or village.
In the mountains, there are nupurikorkamui...... a god who possesses a mountain (bear) and nupuripakorkamui...... a god who possesses the east of the mountain (wolf), an